!! Note on 04/07/2020

1. Each time the model is doing pairing for distances, it is putting too many scenarios into the problem because we should have already combined some scenarios together. (now addressed by adding reduced_node_s)
2. The tolerance check is wrong. It should be using the distance measure that spans the whole modelling period. (Nope, paper uses distance depending on the breakpoint nvm)
3. Distance calculation is wrong. It should be using the distance after merging scenarios. (Nope, it's fine nvm)
4. It doesn't know which p to use for calculating z. (now addressed by calculating p for reduced_node_s)

#### Some information to explain how it works

- First 'breakpoint' is arbitrarily large and we compare scenarios over whole planning period. We delete some scenarios entirely.
This gives you **reduced_s** and **reduced_p**.


- **reduced_s_to_nodes** is defined as **[ scenario, year, node_s ]**. 


- Basically, we keep all scenarios after first 'breakpoint' and stop deleting scenarios. But some scenarios share the same values before certain breakpoints as we merge them. We are really saying for a scenario, in this year we take value from another scenario. This scenario from which we take the value from is called node scenario (**node_s** in the code, meaning node scenario). 


- Say next breakpoint is 2040 in this case, we compare scenarios up to 2039. We know that after merging, ignoring years after breakpoint, there are only certain scenarios remaining from which we read values from. **reduced_node_s** is this reduced pool of **node_s**, whereas **node_s_p** is the probability corresponding to each of the **node_s**. Meanwhile **reduced_s**, the actual number of scenarios, always stays the same. 


- As we reduce the **node_s** pool, we keep updating the third column of **reduced_s_to_nodes** to redefine where a scenario reads its values from. 

#### Need the following: 
- list of breakpoints (i.e. years). Constant.
- mapping between scenario and nodes, with nodes defined by node_year and node_scenario. This needs to be regularly updated.
- a table of scenario vs probability p. Repeatedly updated.
- a tracker of which breakpoints have been reduced. Repeatedly updated.
- a tracker of remaining nodes. Repeatedly updated.
- a mapping between scenario pairs. Repeatedly updated.
- a list of distance between scenarios, with size equal to the mapping between scenarios. Repeatedly updated.
- a generator of scenario values. A function.
- a calculator of distance between two scenarios, compared up to a certain breakpoint. A function.
- a mapping between nodes and breakpoints. Constant.
- a mapping between breakpoints and years. Constant.

#### Inputs
- probability p.
- The input here.
- breakpoints

#### Points to remember:
- we are pruning the nodes
- probability p follows the scenario, although the scenario list is updated repeatedly


In [306]:
import pandas as pd
import numpy as np
from scipy.special import comb
from itertools import combinations
import sys

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=np.inf)

data = pd.read_csv('C:\\Users\\user1\\Desktop\\t_scenDemand.csv')
data.head()

,i_t,i_s,p_scenDemand
0,2020,1,-106.858898
1,2020,2,-61.108668
2,2020,3,-113.257302
3,2020,4,-111.049724
4,2020,5,-201.098780


In [307]:
array = np.array([1, 2, 3, 5])
for s in array:
    print(s)

1
2
3
5


In [308]:
class ScenarioReductionTool:
    
    def __init__(self, input_data, input_breakpts, input_p, tol):
        # define constant inputs
        # note the i_s in data needs to be treated as the node scenario
        self.data = input_data
        self.breakpts = input_breakpts
        self.p = input_p
        self.tol = tol
        self.years = self.data.i_t.unique()
        self.nodes = self.data[['i_t', 'i_s']].values
        self.scenarios = self.data.i_s.unique()
        
        # generate mapping of year to breakpt
        self.year_to_breakpt = self.years
        self.year_to_breakpt = np.c_[self.year_to_breakpt, 
                                     np.zeros(len(self.year_to_breakpt))]
        
        for x in range(self.year_to_breakpt.shape[0]):
            year = self.year_to_breakpt[x, 0]
            
            breakpt_try = year + 1
            while year >= self.years.min(): 
                breakpt_try = breakpt_try - 1
                if breakpt_try in self.breakpts:
                    self.year_to_breakpt[x, 1] = breakpt_try
                    break
        
        # generate mapping between nodes and breakpoints
        breakpts_list = [self.year_to_breakpt[self.year_to_breakpt[:, 0] == x, 1].tolist()[0] for x in self.nodes[:, 0]]
        self.breakpts_to_nodes = np.c_[breakpts_list, self.nodes]
        
        # define inputs that change depending on loop
        # generate mapping between scenario and nodes
        self.s_to_nodes = np.c_[self.data['i_s'].values, self.nodes]
        
        # for outer loop over breakpoints
        self.current_breakpt = 10000
        self.reduced_s = self.scenarios
        self.reduced_node_s = self.scenarios
        self.reduced_nodes = self.nodes
        self.reduced_p = self.p
        self.reduced_node_s_p = self.p
        self.reduced_s_to_nodes = self.s_to_nodes
        
        # for inner loop over scen reduction iterations
        self.current_remaining_node_s = np.zeros(shape=[0])
        self.current_deleted_node_s = np.zeros(shape=[0])
        self.current_s_pairings = None
        self.current_paired_distances = None
        self.c = None
        self.c_pairings = None
        self.closest_s = np.zeros(shape=[0])
        self.z = None
        self.check_tol = None

        
    def set_new_breakpoint(self, set_breakpt):
        """
        Updated attributes:
          - current_breakpt
          - current_s_pairings
        """
        self.current_breakpt = set_breakpt
        self.current_deleted_node_s = np.zeros(shape=[0])
        self.current_remaining_node_s = self.reduced_node_s.copy()
        self.current_s_pairings = self.create_pairings()
        self.current_paired_distances = None
        self.c = None
        self.c_pairings = None
        self.closest_s = np.zeros(shape=[0])
        self.z = None
        self.check_tol = None

    
    def create_pairings(self):
        """
        Create pairings based on reduced scenarios
        
        """
        self.current_s_pairings = np.array(list(combinations(self.reduced_node_s, 2)))

    
    def calc_distances(self):
        """
        Fill attribute current_paired_distances based on self.current_s_pairings.
        Note computationally intensive.
        """
        self.current_paired_distances = np.array(
            [
                np.linalg.norm(
                    self.data[np.logical_and(self.data.i_s == self.current_s_pairings[i, 0], self.data.i_t < self.current_breakpt)].values 
                    - self.data[np.logical_and(self.data.i_s == self.current_s_pairings[i, 1], self.data.i_t < self.current_breakpt)].values
                )
                for i in range(self.current_s_pairings.shape[0])
            ]
        )

    
    def calc_c_and_z(self):
        # for each remaining node s, find min paired distance 
        # where one scenario is the node s,
        # the other one is in the deleted list of node s
        if self.current_deleted_node_s.shape[0] == 0:
            self.c = np.array([
                np.min(
                    self.current_paired_distances[
                        np.logical_or(
                            self.current_s_pairings[:, 0] == l, 
                            self.current_s_pairings[:, 1] == l
                        )
                    ]
                )
                for l in self.current_remaining_node_s
            ])
            
            self.z = np.multiply(self.reduced_node_s_p, self.c)
        else:
            self.c_pairings = np.zeros(shape=[(self.current_deleted_node_s.shape[0]+1) * self.current_remaining_node_s.shape[0], 2])
            i = 0
            for l in self.current_remaining_node_s:
                for k in np.append(self.current_deleted_node_s, l):
                    self.c_pairings[i, 0] = k
                    self.c_pairings[i, 1] = l
                    i += 1

            self.c = np.zeros(shape=self.c_pairings.shape[0])
            for i in range(self.c_pairings.shape[0]): #
                k = self.c_pairings[i, 0]
                l = self.c_pairings[i, 1]
                j = np.delete(self.current_remaining_node_s, np.argwhere(self.current_remaining_node_s == l))
#                 if self.current_breakpt==2045:
#                     print('k = ' + str(k))
#                     print('l = ' + str(l))
#                     print('j:')
#                     print(j)
    
    
#                 pairing_candidates = self.current_s_pairings[
#                     # taking both combinations has no consequence as we are finding the min
#                     np.logical_or(
#                         np.logical_and(
#                             self.current_s_pairings[:, 0] == self.k
#                             np.in1d(self.current_s_pairings[:, 1], self.j)
#                         ),
#                         np.logical_and(
#                             self.current_s_pairings[:, 1] == self.k,
#                             np.in1d(self.current_s_pairings[:, 0], self.j)
#                         )
#                     )
#                 ]

#                 print('pairing candidates')
#                 print(pairing_candidates)
                distance_candidates = self.current_paired_distances[
                    # taking both combinations has no consequence as we are finding the min
                    np.logical_or(
                        np.logical_and(
                            self.current_s_pairings[:, 0] == k,
                            np.in1d(self.current_s_pairings[:, 1], j)
                        ),
                        np.logical_and(
                            self.current_s_pairings[:, 1] == k,
                            np.in1d(self.current_s_pairings[:, 0], j)
                        )
                    )
                ]
#                 print('distance candidates')
#                 print(distance_candidates)
                self.c[i] = np.min(distance_candidates)
        
            self.z = np.zeros(shape=[self.current_remaining_node_s.shape[0]])
            for i in range(self.current_remaining_node_s.shape[0]):
                l = self.current_remaining_node_s[i]
                #!!!
                self.z[i] = np.sum(
                        self.reduced_node_s_p[np.in1d(self.reduced_node_s, np.append(self.current_deleted_node_s, l))]
                        * self.c[
                            np.logical_and(
                                np.in1d(self.c_pairings[:, 0], np.append(self.current_deleted_node_s, l)),
                                self.c_pairings[:, 1] == l
                            )
                        ]
                    )
                                
    
    def calc_tol(self):
        """
        Calculate distance and check tolerance. To do this, current deleted s and remaining s must be defined.
        """
        # get pairing of deleted and remaining scenarios
#         remained_deleted_pairings = np.zeros(shape=[self.current_deleted_node_s.shape[0] * self.current_remaining_node_s.shape[0], 2])
#         x = 0
#         for i in self.current_deleted_node_s:
#             for j in self.current_remaining_node_s:
#                 remained_deleted_pairings[x, 0] = i
#                 remained_deleted_pairings[x, 1] = j
#                 x += 1
        
        # get min distance of pairings between deleted and remaining scenarios for each deleted scenario
        min_paired_distance_for_deleted = np.zeros(shape=[self.current_deleted_node_s.shape[0]])
        for i in np.nditer(self.current_deleted_node_s):
            paired_distance_for_deleted = self.current_paired_distances[
                np.logical_or(
                    np.logical_and(
                        self.current_s_pairings[:, 0] == i,
                        np.in1d(self.current_s_pairings[:, 1], self.current_remaining_node_s),
                    ),
                    np.logical_and(
                        self.current_s_pairings[:, 1] == i,
                        np.in1d(self.current_s_pairings[:, 0], self.current_remaining_node_s),
                    )
                )
            ]
            min_paired_distance_for_deleted[np.argwhere(self.current_deleted_node_s == i)] = np.min(paired_distance_for_deleted)
        
        #!!!
        deleted_p = self.reduced_node_s_p[np.in1d(self.reduced_node_s, self.current_deleted_node_s)]
        self.check_tol = np.sum(np.multiply(deleted_p, min_paired_distance_for_deleted))
        


    def delete_node_s(self):
        """
        Assign a node scenario to list of deleted and remove it from remaining nodes.
        Only ok after z is calculated.
        The updated attributes are:
          - current_deleted_node_s
          - current_remaining_node_s
          - closest_s
        """
        node_s_to_delete = self.current_remaining_node_s[np.argmin(self.z)]
        self.current_deleted_node_s = np.append(self.current_deleted_node_s, node_s_to_delete)
#         if self.current_breakpt not in self.breakpts:
#             np.delete(self.reduced_p, np.argwhere(self.current_remaining_node_s == node_s_to_delete)
                      
        self.current_remaining_node_s = np.delete(
            self.current_remaining_node_s, 
            np.argwhere(self.current_remaining_node_s == node_s_to_delete)
        )
        
        # find scenario the deleted one to merge with, this becomes self.closest_s
        deletion_pairs_candidates = self.current_s_pairings[
            np.logical_or(    
                np.logical_and(
                    self.current_s_pairings[:, 0] == node_s_to_delete, 
                    np.in1d(self.current_s_pairings[:, 1], self.current_remaining_node_s)
                ),
                np.logical_and(
                    self.current_s_pairings[:, 1] == node_s_to_delete, 
                    np.in1d(self.current_s_pairings[:, 0], self.current_remaining_node_s)
                )
            )
        ]
        deletion_pair_distances = self.current_paired_distances[
            np.logical_or(    
                np.logical_and(
                    self.current_s_pairings[:, 0] == node_s_to_delete, 
                    np.in1d(self.current_s_pairings[:, 1], self.current_remaining_node_s)
                ),
                np.logical_and(
                    self.current_s_pairings[:, 1] == node_s_to_delete, 
                    np.in1d(self.current_s_pairings[:, 0], self.current_remaining_node_s)
                )
            )            
        ]
        deletion_pair = deletion_pairs_candidates[np.argmin(deletion_pair_distances)]
        closest_s = deletion_pair[deletion_pair != node_s_to_delete]
        self.closest_s = np.append(self.closest_s, closest_s)
    
        
    def calc_reduced_node_s_p(self):
        """
        Calculate reduced_node_s_p based on reduced_p depending on reduced_node_s.
        returns an array of p with each element correspond to reduced_node_s in the same position.
        Should be updated when scenarios are actually deleted from reduced_node_s.
        Should be updated only when s_to_nodes is updated.
        """
        self.reduced_node_s_p = np.zeros(shape=[self.reduced_node_s.shape[0]])
        for i in range(self.reduced_node_s.shape[0]):
            reduced_s_with_reduced_node_s = self.reduced_s_to_nodes[
                np.logical_and(
                    self.reduced_s_to_nodes[:, 1] == self.current_breakpt - 1 ,
                    self.reduced_s_to_nodes[:, 2] == self.reduced_node_s[i]
                )
            ]
            reduced_s_with_reduced_node_s = np.unique(reduced_s_with_reduced_node_s[:, 0])
            print('reduced_node_s='+str(self.reduced_node_s[i]))
            print('reduced_s_with_reduced_node_s=')
            print(reduced_s_with_reduced_node_s)
            
            
            self.reduced_node_s_p[i] = np.sum(self.reduced_p[np.in1d(self.reduced_s, reduced_s_with_reduced_node_s)])
    
    
    def delete_scenarios_nodes(self):
        """
        For 'first breakpoint', delete entire scenarios and all nodes. This forms the reduced_s.
        Note probability also need merging in this step.
        After that, replace all nodes indexed with deleted scenarios before current breakpoint
        with closest scenarios. 
        """
        deleted_closest_s_concat = np.c_[self.current_deleted_node_s, self.closest_s]
        
        if self.current_breakpt not in self.breakpts:
            # if current breakpt not in breakpt, we are in first loop and 
            # card(p) == card(reduced_p) == card(scenarios) == card(reduced_s)
            
            for i in range(self.p.shape[0]):
                # if scenario is in deleted scenarios identify the closest scenario and add prob to it
                if self.scenarios[i] in self.current_deleted_node_s:
                    deleted_node_s = self.scenarios[i]
                    closest_s = deleted_closest_s_concat[deleted_closest_s_concat[:, 0] == deleted_node_s][0, 1]
                    closest_s_index = np.argwhere(self.scenarios == closest_s)
                    self.reduced_p[closest_s_index] += self.p[self.scenarios == deleted_node_s][0]
            
            self.reduced_p = np.delete(self.reduced_p, np.argwhere(np.in1d(self.scenarios, self.current_deleted_node_s)))
            self.reduced_node_s_p = self.reduced_p
            self.reduced_s = np.delete(self.reduced_s, np.argwhere(np.in1d(self.scenarios, self.current_deleted_node_s)))
            self.reduced_node_s = np.delete(self.reduced_node_s, np.argwhere(np.in1d(self.scenarios, self.current_deleted_node_s)))
            self.reduced_s_to_nodes = np.delete(
                self.reduced_s_to_nodes, 
                np.argwhere(np.in1d(self.s_to_nodes[:, 0], self.current_deleted_node_s)), axis=0
            )
            
        else:
            np.delete(self.reduced_nodes, np.argwhere(
                np.logical_and(
                    np.in1d(self.reduced_nodes[1], self.current_deleted_node_s),
                    self.reduced_nodes[0] < self.current_breakpt
            )))
            
            self.reduced_node_s = np.delete(self.reduced_node_s, np.argwhere(np.in1d(self.reduced_node_s, self.current_deleted_node_s)))
                        
            for i in np.nditer(self.current_deleted_node_s):
                closest_s = deleted_closest_s_concat[deleted_closest_s_concat[:, 0] == i][0, 1]
                self.reduced_s_to_nodes[:, 2] = np.where(
                    np.logical_and(
                        self.reduced_s_to_nodes[:, 2] == i,
                        self.reduced_s_to_nodes[:, 1] < self.current_breakpt
                    ),
                    closest_s, 
                    self.reduced_s_to_nodes[:, 2]
                )
                
            self.calc_reduced_node_s_p()
        
    
    def ops_over_breakpt(self):
        """
        Do scenario reduction until tolerance is reached.
        """
        i = 1
        while i < self.scenarios.shape[0]:
            previous_deleted_node_s = np.copy(self.current_deleted_node_s)
            previous_remaining_node_s = np.copy(self.current_remaining_node_s)
            previous_closest_s = np.copy(self.closest_s)
            
            print('loop ' + str(i) + ' of breakpoint ' + str(self.current_breakpt))
            self.calc_c_and_z()
            self.delete_node_s()
            
            print('Deleted scenarios = ')
            print(self.current_deleted_node_s)
            
            print('Paired scenarios = ')
            print(self.closest_s)
            
            self.calc_tol()
            print('tol = ' + str(self.check_tol) + ', limit = ' + str(self.tol))
            
            if self.check_tol >= self.tol:
                
                print('Tol reached, stop deleting scenarios')
                self.current_deleted_node_s = previous_deleted_node_s
                self.current_remaining_node_s = previous_remaining_node_s
                self.closest_s = previous_closest_s
                break
                
            elif i == self.scenarios.shape[0]:
                print('Max iteration reached for breakpoint.')
                
            else:
                i += 1
            
        print('Deleting scenarios...')
        self.delete_scenarios_nodes()
        print('Final remaining scenarios:')
        print(self.reduced_node_s)
        print('Final remaining probabilities:')
        print(self.reduced_node_s_p)
        print('Final scenario tree:')
        for i in range(self.reduced_s_to_nodes.shape[0]):
            print(self.reduced_s_to_nodes[i])

            
    def ops_over_all_breakpoints(self):
        for i in reversed(np.r_[np.delete(self.breakpts, 0), 10000]):
            self.set_new_breakpoint(i)
            self.create_pairings()
            self.calc_distances()
            self.ops_over_breakpt()   
        
        

In [309]:
breakpts = [2020, 2025, 2030, 2035, 2040, 2045]
p = np.ones(100) * 0.01
ScenTree = ScenarioReductionTool(data, breakpts, p, 100)
ScenTree.ops_over_all_breakpoints()

loop 1 of breakpoint 10000
Deleted scenarios = 
[4.]
Paired scenarios = 
[11.]
tol = 12.262412382628725, limit = 100
loop 2 of breakpoint 10000
Deleted scenarios = 
[ 4. 53.]
Paired scenarios = 
[11. 70.]
tol = 25.84116455654848, limit = 100
loop 3 of breakpoint 10000
Deleted scenarios = 
[ 4. 53. 45.]
Paired scenarios = 
[11. 70. 68.]
tol = 39.6136515738018, limit = 100
loop 4 of breakpoint 10000
Deleted scenarios = 
[ 4. 53. 45. 65.]
Paired scenarios = 
[11. 70. 68. 11.]
tol = 53.88393145499419, limit = 100
loop 5 of breakpoint 10000
Deleted scenarios = 
[ 4. 53. 45. 65. 34.]
Paired scenarios = 
[11. 70. 68. 11. 80.]
tol = 68.30382827310936, limit = 100
loop 6 of breakpoint 10000
Deleted scenarios = 
[  4.  53.  45.  65.  34. 100.]
Paired scenarios = 
[11. 70. 68. 11. 80. 70.]
tol = 82.73507352105113, limit = 100
loop 7 of breakpoint 10000
Deleted scenarios = 
[  4.  53.  45.  65.  34. 100.   5.]
Paired scenarios = 
[11. 70. 68. 11. 80. 70. 20.]
tol = 97.32142862167535, limit = 100
l

[  51 2034   51]
[  52 2034   52]
[  54 2034   54]
[  55 2034   55]
[  56 2034   56]
[  57 2034   57]
[  58 2034   58]
[  59 2034   59]
[  60 2034   60]
[  61 2034   61]
[  62 2034   62]
[  63 2034   63]
[  64 2034   64]
[  66 2034   66]
[  67 2034   67]
[  68 2034   68]
[  69 2034   69]
[  70 2034   70]
[  71 2034   71]
[  72 2034   72]
[  73 2034   73]
[  74 2034   74]
[  75 2034   75]
[  76 2034   76]
[  77 2034   77]
[  78 2034   78]
[  79 2034   79]
[  80 2034   80]
[  81 2034   81]
[  82 2034   82]
[  83 2034   83]
[  84 2034   84]
[  85 2034   85]
[  86 2034   86]
[  87 2034   87]
[  88 2034   88]
[  89 2034   89]
[  90 2034   90]
[  91 2034   91]
[  92 2034   92]
[  93 2034   93]
[  94 2034   94]
[  95 2034   95]
[  96 2034   96]
[  97 2034   97]
[  98 2034   98]
[  99 2034   99]
[   1 2035    1]
[   2 2035    2]
[   3 2035    3]
[   6 2035    6]
[   7 2035    7]
[   8 2035    8]
[   9 2035    9]
[  10 2035   10]
[  11 2035   11]
[  12 2035   12]
[  13 2035   13]
[  14 2035   1

[  40 2048   40]
[  41 2048   41]
[  42 2048   42]
[  43 2048   43]
[  44 2048   44]
[  46 2048   46]
[  47 2048   47]
[  48 2048   48]
[  49 2048   49]
[  50 2048   50]
[  51 2048   51]
[  52 2048   52]
[  54 2048   54]
[  55 2048   55]
[  56 2048   56]
[  57 2048   57]
[  58 2048   58]
[  59 2048   59]
[  60 2048   60]
[  61 2048   61]
[  62 2048   62]
[  63 2048   63]
[  64 2048   64]
[  66 2048   66]
[  67 2048   67]
[  68 2048   68]
[  69 2048   69]
[  70 2048   70]
[  71 2048   71]
[  72 2048   72]
[  73 2048   73]
[  74 2048   74]
[  75 2048   75]
[  76 2048   76]
[  77 2048   77]
[  78 2048   78]
[  79 2048   79]
[  80 2048   80]
[  81 2048   81]
[  82 2048   82]
[  83 2048   83]
[  84 2048   84]
[  85 2048   85]
[  86 2048   86]
[  87 2048   87]
[  88 2048   88]
[  89 2048   89]
[  90 2048   90]
[  91 2048   91]
[  92 2048   92]
[  93 2048   93]
[  94 2048   94]
[  95 2048   95]
[  96 2048   96]
[  97 2048   97]
[  98 2048   98]
[  99 2048   99]
[   1 2049    1]
[   2 2049    

[  28 2033   28]
[  29 2033   29]
[  30 2033   30]
[  31 2033   31]
[  32 2033   32]
[  33 2033   33]
[  35 2033   35]
[  36 2033   36]
[  37 2033   37]
[  38 2033   38]
[  39 2033   39]
[  40 2033   80]
[  41 2033   41]
[  42 2033   42]
[  43 2033   43]
[  44 2033   44]
[  46 2033   46]
[  47 2033   47]
[  48 2033   48]
[  49 2033   49]
[  50 2033   50]
[  51 2033   51]
[  52 2033   52]
[  54 2033   54]
[  55 2033   55]
[  56 2033   56]
[  57 2033   57]
[  58 2033   58]
[  59 2033   59]
[  60 2033   60]
[  61 2033   61]
[  62 2033   62]
[  63 2033   64]
[  64 2033   64]
[  66 2033   66]
[  67 2033   67]
[  68 2033   68]
[  69 2033   69]
[  70 2033   70]
[  71 2033   71]
[  72 2033   72]
[  73 2033   73]
[  74 2033   74]
[  75 2033   75]
[  76 2033   70]
[  77 2033   70]
[  78 2033   90]
[  79 2033   79]
[  80 2033   80]
[  81 2033   81]
[  82 2033   82]
[  83 2033   83]
[  84 2033   84]
[  85 2033   85]
[  86 2033   86]
[  87 2033   87]
[  88 2033   88]
[  89 2033   89]
[  90 2033   9

[  72 2046   72]
[  73 2046   73]
[  74 2046   74]
[  75 2046   75]
[  76 2046   76]
[  77 2046   77]
[  78 2046   78]
[  79 2046   79]
[  80 2046   80]
[  81 2046   81]
[  82 2046   82]
[  83 2046   83]
[  84 2046   84]
[  85 2046   85]
[  86 2046   86]
[  87 2046   87]
[  88 2046   88]
[  89 2046   89]
[  90 2046   90]
[  91 2046   91]
[  92 2046   92]
[  93 2046   93]
[  94 2046   94]
[  95 2046   95]
[  96 2046   96]
[  97 2046   97]
[  98 2046   98]
[  99 2046   99]
[   1 2047    1]
[   2 2047    2]
[   3 2047    3]
[   6 2047    6]
[   7 2047    7]
[   8 2047    8]
[   9 2047    9]
[  10 2047   10]
[  11 2047   11]
[  12 2047   12]
[  13 2047   13]
[  14 2047   14]
[  15 2047   15]
[  16 2047   16]
[  17 2047   17]
[  18 2047   18]
[  19 2047   19]
[  20 2047   20]
[  21 2047   21]
[  22 2047   22]
[  23 2047   23]
[  24 2047   24]
[  25 2047   25]
[  26 2047   26]
[  27 2047   27]
[  28 2047   28]
[  29 2047   29]
[  30 2047   30]
[  31 2047   31]
[  32 2047   32]
[  33 2047   3

...

Deleted scenarios = 
[22. 26. 59. 84. 79. 27. 72. 13. 32. 33. 49. 43. 97.  9. 12. 86.]
Paired scenarios = 
[47. 91. 80. 85. 69. 38. 11. 73. 47. 60. 47. 68. 69. 70. 62. 85.]
tol = 64.45029962216537, limit = 100
loop 17 of breakpoint 2030
Deleted scenarios = 
[22. 26. 59. 84. 79. 27. 72. 13. 32. 33. 49. 43. 97.  9. 12. 86.  3.]
Paired scenarios = 
[47. 91. 80. 85. 69. 38. 11. 73. 47. 60. 47. 68. 69. 70. 62. 85. 47.]
tol = 69.17265351814511, limit = 100
loop 18 of breakpoint 2030
Deleted scenarios = 
[22. 26. 59. 84. 79. 27. 72. 13. 32. 33. 49. 43. 97.  9. 12. 86.  3. 31.]
Paired scenarios = 
[47. 91. 80. 85. 69. 38. 11. 73. 47. 60. 47. 68. 69. 70. 62. 85. 47. 69.]
tol = 73.91536471251507, limit = 100
loop 19 of breakpoint 2030
Deleted scenarios = 
[22. 26. 59. 84. 79. 27. 72. 13. 32. 33. 49. 43. 97.  9. 12. 86.  3. 31. 57.]
Paired scenarios = 
[47. 91. 80. 85. 69. 38. 11. 73. 47. 60. 47. 68. 69. 70. 62. 85. 47. 69. 85.]
tol = 78.7116031448164, limit = 100
loop 20 of breakpoint 2030
Delet

[  29 2036   29]
[  30 2036   30]
[  31 2036   31]
[  32 2036   32]
[  33 2036   33]
[  35 2036   35]
[  36 2036   36]
[  37 2036   89]
[  38 2036   38]
[  39 2036   39]
[  40 2036   80]
[  41 2036   20]
[  42 2036   60]
[  43 2036   43]
[  44 2036   44]
[  46 2036   46]
[  47 2036   47]
[  48 2036   48]
[  49 2036   49]
[  50 2036   50]
[  51 2036   51]
[  52 2036   52]
[  54 2036   54]
[  55 2036   55]
[  56 2036   56]
[  57 2036   57]
[  58 2036   58]
[  59 2036   59]
[  60 2036   60]
[  61 2036   61]
[  62 2036   62]
[  63 2036   64]
[  64 2036   64]
[  66 2036   70]
[  67 2036   69]
[  68 2036   68]
[  69 2036   69]
[  70 2036   70]
[  71 2036   71]
[  72 2036   72]
[  73 2036   73]
[  74 2036   74]
[  75 2036   11]
[  76 2036   70]
[  77 2036   70]
[  78 2036   90]
[  79 2036   79]
[  80 2036   80]
[  81 2036   64]
[  82 2036   82]
[  83 2036   70]
[  84 2036   84]
[  85 2036   85]
[  86 2036   86]
[  87 2036   69]
[  88 2036   88]
[  89 2036   89]
[  90 2036   90]
[  91 2036   9

loop 1 of breakpoint 2025
Deleted scenarios = 
[39.]
Paired scenarios = 
[61.]
tol = 1.4895221468245974, limit = 100
loop 2 of breakpoint 2025
Deleted scenarios = 
[39. 98.]
Paired scenarios = 
[61. 94.]
tol = 3.057272349031199, limit = 100
loop 3 of breakpoint 2025
Deleted scenarios = 
[39. 98. 54.]
Paired scenarios = 
[61. 94. 38.]
tol = 4.758371928716757, limit = 100
loop 4 of breakpoint 2025
Deleted scenarios = 
[39. 98. 54. 51.]
Paired scenarios = 
[61. 94. 38. 38.]
tol = 6.476839995570392, limit = 100
loop 5 of breakpoint 2025
Deleted scenarios = 
[39. 98. 54. 51. 52.]
Paired scenarios = 
[61. 94. 38. 38. 88.]
tol = 8.499038477318713, limit = 100
loop 6 of breakpoint 2025
Deleted scenarios = 
[39. 98. 54. 51. 52.  2.]
Paired scenarios = 
[61. 94. 38. 38. 88. 69.]
tol = 10.531909412395402, limit = 100
loop 7 of breakpoint 2025
Deleted scenarios = 
[39. 98. 54. 51. 52.  2. 44.]
Paired scenarios = 
[61. 94. 38. 38. 88. 69. 91.]
tol = 13.127881702803503, limit = 100
loop 8 of breakpo

[  19 2023   19]
[  20 2023   20]
[  21 2023   69]
[  22 2023   47]
[  23 2023   20]
[  24 2023   80]
[  25 2023   70]
[  26 2023   85]
[  27 2023   38]
[  28 2023   11]
[  29 2023   80]
[  30 2023   60]
[  31 2023   69]
[  32 2023   47]
[  33 2023   60]
[  35 2023   60]
[  36 2023   73]
[  37 2023   80]
[  38 2023   38]
[  39 2023   85]
[  40 2023   80]
[  41 2023   20]
[  42 2023   60]
[  43 2023   68]
[  44 2023   85]
[  46 2023   11]
[  47 2023   47]
[  48 2023   70]
[  49 2023   47]
[  50 2023   73]
[  51 2023   38]
[  52 2023   20]
[  54 2023   38]
[  55 2023   69]
[  56 2023   73]
[  57 2023   85]
[  58 2023   80]
[  59 2023   80]
[  60 2023   60]
[  61 2023   85]
[  62 2023   11]
[  63 2023   73]
[  64 2023   73]
[  66 2023   70]
[  67 2023   69]
[  68 2023   68]
[  69 2023   69]
[  70 2023   70]
[  71 2023   73]
[  72 2023   11]
[  73 2023   73]
[  74 2023   73]
[  75 2023   11]
[  76 2023   70]
[  77 2023   70]
[  78 2023   69]
[  79 2023   69]
[  80 2023   80]
[  81 2023   7

[  56 2037   56]
[  57 2037   57]
[  58 2037   58]
[  59 2037   59]
[  60 2037   60]
[  61 2037   61]
[  62 2037   62]
[  63 2037   64]
[  64 2037   64]
[  66 2037   70]
[  67 2037   69]
[  68 2037   68]
[  69 2037   69]
[  70 2037   70]
[  71 2037   71]
[  72 2037   72]
[  73 2037   73]
[  74 2037   74]
[  75 2037   11]
[  76 2037   70]
[  77 2037   70]
[  78 2037   90]
[  79 2037   79]
[  80 2037   80]
[  81 2037   64]
[  82 2037   82]
[  83 2037   70]
[  84 2037   84]
[  85 2037   85]
[  86 2037   86]
[  87 2037   69]
[  88 2037   88]
[  89 2037   89]
[  90 2037   90]
[  91 2037   91]
[  92 2037   70]
[  93 2037   69]
[  94 2037   94]
[  95 2037   95]
[  96 2037   96]
[  97 2037   97]
[  98 2037   98]
[  99 2037   99]
[   1 2038   47]
[   2 2038    2]
[   3 2038    3]
[   6 2038    6]
[   7 2038   70]
[   8 2038    8]
[   9 2038    9]
[  10 2038   10]
[  11 2038   11]
[  12 2038   12]
[  13 2038   13]
[  14 2038   14]
[  15 2038   15]
[  16 2038   16]
[  17 2038   17]
[  18 2038   7

In [317]:
np.unique(ScenTree.reduced_s_to_nodes[ScenTree.reduced_s_to_nodes[:,1]==2024][:,2])

array([11, 19, 20, 38, 47, 60, 68, 69, 70, 73, 80, 85], dtype=int64)

In [320]:
np.sum(ScenTree.reduced_node_s_p)

1.0000000000000002

In [310]:
# ScenTree.reduced_s_to_nodes[ScenTree.reduced_s_to_nodes[:, 0]==10]

In [311]:
# breakpts = [2020, 2025, 2030, 2035, 2040, 2045]
# p = np.ones(100) * 0.01
# ScenTree = ScenarioReductionTool(data, breakpts, p, 100)
# ScenTree.set_new_breakpoint(10000)
# ScenTree.create_pairings()
# ScenTree.calc_distances()
# ScenTree.ops_over_breakpt()

In [312]:
# ScenTree.set_new_breakpoint(2045)
# ScenTree.create_pairings()
# ScenTree.calc_distances()
# ScenTree.ops_over_breakpt()